# 02-02 - Azure Cosmos DB in deiner Resource Group erstellen

In dieser Anleitung zeigen wir dir Schritt für Schritt, wie du eine neue Azure Cosmos DB Instanz in deiner Resource Group erstellst. Wir nutzen dabei **Azure Cosmos DB für NoSQL**, da es für unsere Zwecke am besten geeignet ist.

### Eine neue Azure Cosmos DB Instanz erstellen
Öffne das Azure Portal und suche im **Marketplace** nach **Cosmos DB**. Wähle den entsprechenden Eintrag aus.  

![CosmosDB suchen](images/find-cosmosdb.png)

### NoSQL als API-Typ wählen
Im Schritt „Empfohlene APIs“ wähle **Azure Cosmos DB für NoSQL**.

#### ❓ Was ist NoSQL?
NoSQL-Datenbanken sind flexible, schemafreie Datenbanken, die besonders gut für große, verteilte Anwendungen geeignet sind. Im Gegensatz zu relationalen SQL-Datenbanken benötigen NoSQL-Datenbanken keine starren Tabellenstrukturen und sind optimal für dynamische und skalierbare Anwendungen.

### Einrichten der Instanz
- **Account-Name:** Wähle einen **eindeutigen Namen** für deine Cosmos DB Instanz.
- **Region:** Wähle **Switzerland North** als Standort, wenn du die Ressource für die Bund-Projekte erstellst.

![CosmosDB erstellen](images/create-cosmosdb.png)

### Availability Zones deaktivieren
Setze **Availability Zones** auf **Deaktiviert**, da wir hier nur **Nicht-Produktionsdaten** speichern. Falls die Daten verloren gehen, ist der Schaden gering. Für **Produktionsumgebungen** solltest du **Aktivieren** wählen, um eine höhere Verfügbarkeit sicherzustellen.

### Kapazitätsmodus: Serverless wählen
Wähle **Serverless** als Kapazitätsmodus.

#### ❓ Warum nicht „Provisioned Throughput“?
- In einem intensiven **Hackathon** haben wir **klare Spitzenzeiten**, in denen die Datenbank stark genutzt wird.
- In anderen Zeiten benötigen wir den Service kaum oder gar nicht.
- **Serverless** erlaubt es uns, nur für die tatsächliche Nutzung zu zahlen, anstatt kontinuierliche Kapazitäten bereitzustellen.

### Netzwerkkonfiguration
Im **Netzwerk-Tab** wähle **Datenverkehr von allen Netzwerken zulassen**, damit jeder Teilnehmer auf die Datenbank zugreifen kann.

### Service erstellen
Lasse alle weiteren Einstellungen unverändert und **erstelle die Cosmos DB Instanz**.

Nach einigen Minuten ist die Instanz einsatzbereit!

---



### Anmeldung bei Azure in der Konsole

Um sich bei Azure über die Konsole anzumelden, verwenden Sie folgenden Befehl:

```sh
az login
```

Falls Sie mehrere Mandanten (Tenants) haben und sich gezielt bei einem bestimmten anmelden möchten, verwenden Sie:

```sh
az login --tenant <TENANT_ID>
```

Beispiel:

```sh
az login --tenant 119cbc86-5275-4878-8321-4d8da34a0dc2
```

Dadurch wird sichergestellt, dass Sie sich direkt in der gewünschten Azure Active Directory (AAD) Umgebung anmelden.

### Öffentlichen Netzwerkzugang aktivieren & Zugriff mit Primär-/Sekundärschlüssel ermöglichen  

Dieser Schritt stellt sicher, dass Ihre Azure CosmosDB **über das öffentliche Internet erreichbar ist** und **mit Primär-/Sekundärschlüsseln** darauf zugegriffen werden kann.

Ersetzen Sie die **Platzhalter** durch Ihre tatsächlichen Werte und führen Sie die Befehle nacheinander aus:

Überprüfen, ob die lokale Authentifizierung aktiviert ist. Sobald disableLocalAuth=false ist, können Sie Ihre Primär-/Sekundärschlüssel verwenden, um auf die CosmosDB zuzugreifen.
```sh
az cosmosdb show \
    --name "<COSMOSDB_NAME>" \
    --resource-group "<RESOURCE_GROUP>" \
    --query "disableLocalAuth"
```

Aktivieren des öffentlichen Netzwerkzugangs
```sh
az cosmosdb update \
    --name "<COSMOSDB_NAME>" \
    --resource-group "<RESOURCE_GROUP>" \
    --public-network-access ENABLED
```



#### Erstellen einer neuen Datenbank und eines Containers in Azure CosmosDB

1. Navigiere im Azure-Portal zur CosmosDB-Instanz.
2. Klicke auf **Data Explorer** im linken Menü.
3. Wähle **Neuer Container**:
   - **Datenbank-ID**: wir vergeben einen Namen für deine neue Datenbank.
   - **Container-ID**: wir vergeben einen Namen für deinen Container.
   - **Partition Key**: wir wählen einen Partition Key, z. B. `/id` oder eine andere geeignete Spalte für dein Schema.
   - **Unique Key**: wir wählen einen Unique Key, z. B. `/id` oder eine passende Spalte für dein Schema.
   - Klicke auf **OK**, um den Container zu erstellen.

![Neue Datenbank und Container erstellen](images/new-container.png)

#### Verbindung mit Umgebungsvariablen herstellen

Nachdem wir die nötigen Werte im Portal gefunden haben, können wir diese als **Umgebungsvariablen** für unsere Anwendung setzen (.env Datei überarbeiten).

Der Schlüssel und URL findest du im **Keys**-Tab.
![Primärschlüssel](images/key-and-url.png)

```bash
COSMOSDB_KEY="<PRIMÄRER_SCHLÜSSEL_AUS_AZURE_PORTAL>"
COSMOSDB_URL="<URI_AUS_AZURE_PORTAL>"
```

Diese Werte entsprechen den Werten, die du gerade im Data Explorer Tab eingegeben hast.

```bash
COSMOSDB_NAME="<NAME_DER_ERSTELLTEN_DATENBANK>"
COSMOSDB_CONTAINER_NAME="<NAME_DES_ERSTELLTEN_CONTAINERS>"
```


Der Code unten sollte nun **CosmosDB connection successful and container initialized!** zurückschreiben.

In [17]:
# Import the required libraries
from azure.cosmos import CosmosClient, PartitionKey

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Azure CosmosDB Connection Details
COSMOS_DB_URL = os.getenv("COSMOSDB_URL")
COSMOS_DB_KEY = os.getenv("COSMOSDB_KEY")
DATABASE_NAME = os.getenv("COSMOSDB_NAME")
CONTAINER_NAME = os.getenv("COSMOSDB_CONTAINER_NAME")

# Connect to CosmosDB
client = CosmosClient(COSMOS_DB_URL, COSMOS_DB_KEY)
database = client.create_database_if_not_exists(DATABASE_NAME)

# You can also create the container via the lines of code below, just like what we did on the Azure Portal
container = database.create_container_if_not_exists(
    id=CONTAINER_NAME,
    partition_key=PartitionKey(path="/id"),  
)

print("CosmosDB connection successful and container initialized!")

CosmosDB connection successful and container initialized!


#### Generierung von Embeddings für unsere Filmdaten

Mit `AzureOpenAIEmbeddings` können wir Embeddings für Textdaten mit Azure OpenAI generieren. Die Instanz kann verwendet werden, um Text in Vektoren umzuwandeln, die in verschiedenen NLP-Anwendungen genutzt werden können.


In [18]:
from langchain_openai import AzureOpenAIEmbeddings
import os
azure_openai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
)

In [19]:
def generate_embedding(content):
    """
    Generates an embedding for the given content using Azure OpenAI.
    """
    try:
        return azure_openai_embeddings.embed_query(content)  # Ensure this returns a list of floats
    except Exception as e:
        print(f"Error generating embedding: {e}")
        return []  # Return an empty list if embedding fails

#### Laden von Filmdaten mit `CSVLoader` aus der `langchain`-Bibliothek.

Mit der `CSVLoader`-Klasse aus der `langchain`-Bibliothek können wir Daten aus einer CSV-Datei einfach laden und für die Verarbeitung in NLP-Anwendungen vorbereiten. In diesem Beispiel wird der Code verwendet, um Filmdaten aus einer CSV-Datei zu laden, wobei der Loader die Struktur der Datei automatisch erkennt und die Daten in ein passendes Format umwandelt. Diese Daten können dann für verschiedene Anwendungen genutzt werden, wie etwa die Erstellung von Datenbanken, Suchindizes oder als Grundlage für maschinelles Lernen.


In [20]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(
    file_path='./movies.csv',
    source_column='original_title',
    encoding='utf-8',
    csv_args={
        'delimiter': ',',
        'fieldnames': [
            'id', 'original_language', 'original_title', 'popularity',
            'release_date', 'genre',
            'overview', 'revenue', 'runtime', 'tagline'
        ]
    }
)
data = loader.load()

# Rather than load all 500 movies into Azure AI search, we will use a
# smaller subset of movie data to make things quicker. The more movies you load,
# the more time it will take for embeddings to be generated.

data = data[1:51]
print('Loaded %s movies.' % len(data))

Loaded 50 movies.


Nun iterieren wir über eine Liste von Filmen und erstellt für jeden Film ein neues strukturiertes Item, das als Dictionary gespeichert wird. Zunächst wird der Inhalt jedes Films durch die Funktion `parse_movie` in einzelne Felder zerlegt und die entsprechenden Daten werden in einem gut strukturierten Format abgelegt. Dabei wird darauf geachtet, dass die Datentypen korrekt umgewandelt werden, z. B. wird die `id` in eine Ganzzahl und das `genre` in eine Liste umgewandelt. Vor dem Parsen wird für jeden Film ein Vektor (Embedding) mit der Funktion `generate_embedding` generiert, um die Daten für Anwendungen im Bereich maschinelles Lernen oder Ähnliches vorzubereiten. Schließlich wird eine Liste von Filmdaten erstellt, die nun mit den generierten Vektoren angereichert sind.


In [21]:
# Loop through all of the movies and create a new item for each one.

def parse_movie(movie, vector):
    """
    Parses a movie object into a structured dictionary with proper data types.
    """
    try:
        content = movie.page_content
        fields = dict(line.split(": ", 1) for line in content.split("\n") if ": " in line)

        # Convert data types correctly
        movie_data = {
            "id": int(float(fields.get("id", "0"))),  # Ensure it's an integer
            "original_language": fields.get("original_language", "").strip(),
            "original_title": fields.get("original_title", "").strip(),
            "popularity": fields.get("popularity", "0"),
            "release_date": fields.get("release_date", "").strip(),
            "vote_average": fields.get("vote_average", "0"),
            "vote_count": fields.get("vote_count", "0"),
            "genre": fields.get("genre", "[]"),  # Convert to a list
            "overview": fields.get("overview", "").strip(),
            "revenue": fields.get("revenue", "0"),
            "runtime": fields.get("runtime", "0"),
            "tagline": fields.get("tagline", "").strip(),
            "vector": vector
        }
        
        return movie_data

    except Exception as e:
        print(f"Error parsing movie: {e}")
        return {}

# Parse and vectorize all movies
parsed_movies = []
for movie in data:
    content = movie.page_content
    vector = generate_embedding(content)  # Generate embedding before parsing
    parsed_movies.append(parse_movie(movie, vector))  # Pass vector to parse_movie

print(parsed_movies[0])  # Print first parsed movie to verify

print(f"New items structure with embeddings created for {len(parsed_movies)} movies.")

{'id': 381284, 'original_language': 'en', 'original_title': 'Hidden Figures', 'popularity': '49.802', 'release_date': '2016-12-10', 'vote_average': '0', 'vote_count': '0', 'genre': '8.1', 'overview': '7310.0', 'revenue': "['Drama', 'History']", 'runtime': 'The untold story of Katherine G. Johnson, Dorothy Vaughan and Mary Jackson – brilliant African-American women working at NASA and serving as the brains behind one of the greatest operations in history – the launch of astronaut John Glenn into orbit. The visionary trio crossed all gender and race lines to inspire generations to dream big.', 'tagline': '230698791.0', 'vector': [-0.03348075598478317, 0.017375215888023376, -0.00873591098934412, 0.008784214034676552, -0.003141408786177635, -0.018851902335882187, 0.007638746406883001, 0.05183583125472069, -0.010875036008656025, 0.03541287034749985, 0.013034862466156483, -0.01817566342651844, 0.010171194560825825, 0.015498306602239609, -0.043389737606048584, 0.016077939420938492, -0.0030120

Als letzter Schritt laden wir die Daten zum Azure Container.
Nach dem Hochladen können wir im Data Explorer die Daten auf einem UI abrufen.

In [ ]:
# Upload Data to CosmosDB
for item in parsed_movies:
    # Ensure 'id' is always a string (CosmosDB requires string IDs)
    item["id"] = str(item["id"])

    try:
        container.upsert_item(item)  # Insert or update if exists
        print(f"Uploaded: {item['original_title']}")
    except Exception as e:
        print(f"Error uploading {item['original_title']}: {e}")

print(f"Successfully uploaded {len(parsed_movies)} movies into CosmosDB! 🚀")

Uploaded: Hidden Figures
Uploaded: Gridlocked
Uploaded: Joker
Uploaded: The Sand
Uploaded: America: The Motion Picture
Uploaded: 僕のヒーローアカデミア THE MOVIE ～2人の英雄～
Uploaded: Under Siege 2: Dark Territory
Uploaded: The Enforcer
Uploaded: Ruby Sparks
Uploaded: They Came Together
Uploaded: The Handmaid's Tale
Uploaded: Броненосец Потёмкин
Uploaded: Enemy of the State
Uploaded: Pistol Whipped
Uploaded: 맛있는 비행
Uploaded: Wheelman
Uploaded: Raising Arizona
Uploaded: Rampage
Uploaded: Evolution
Uploaded: Man on Wire
Uploaded: Work It
Uploaded: Step Sisters
Uploaded: Pirates of the Caribbean: Tales of the Code – Wedlocked
Uploaded: I Don't Know How She Does It
Uploaded: The Wrestler
Uploaded: The Sisterhood of the Traveling Pants
Uploaded: Charlie Wilson's War
Uploaded: El Infierno
Uploaded: Una última y nos vamos
Uploaded: Immortals
Uploaded: Pandorum
Uploaded: Women
Uploaded: Hotel Transylvania 3: Summer Vacation
Uploaded: Alex Strangelove
Uploaded: The Painted Veil
Uploaded: Johnson Family Vacati